In [2]:
#Imports + config
import os
import pandas as pd

STORE_ID = 1
ITEMS = [15, 28, 18, 13, 38]

# Aritzia-style naming 
ITEM_MAP = {
    15: "Super Puff Shorty",
    28: "Effortless Pant",
    18: "Sculpt Knit Tank",
    13: "Wilfred Sweater",
    38: "TNA Sweatfleece Hoodie"}


In [4]:
#Load raw data + basic validation
df = pd.read_csv("train.csv")

print("Raw shape:", df.shape)
print(df.head())
print(df.dtypes)

expected_cols = {"date", "store", "item", "sales"}
missing = expected_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing columns in raw data: {missing}")

# Ensure types
df["date"] = pd.to_datetime(df["date"], errors="raise")
df["store"] = df["store"].astype(int)
df["item"]  = df["item"].astype(int)
df["sales"] = df["sales"].astype(int)

Raw shape: (913000, 4)
         date  store  item  sales
0  2013-01-01      1     1     13
1  2013-01-02      1     1     11
2  2013-01-03      1     1     14
3  2013-01-04      1     1     13
4  2013-01-05      1     1     10
date     object
store     int64
item      int64
sales     int64
dtype: object


In [5]:
#Filter to your pilot scope (store 1 + 5 items)
pilot = df[(df["store"] == STORE_ID) & (df["item"].isin(ITEMS))].copy()
print("Pilot shape:", pilot.shape)

# Add human-readable SKU name
pilot["sku_name"] = pilot["item"].map(ITEM_MAP)

if pilot["sku_name"].isna().any():
    missing_items = pilot.loc[pilot["sku_name"].isna(), "item"].unique()
    raise ValueError(f"ITEM_MAP missing mappings for items: {missing_items}")

Pilot shape: (9130, 4)


In [6]:
#Check completeness (fill missing dates per SKU)
# Create a complete daily date range
full_dates = pd.date_range(pilot["date"].min(), pilot["date"].max(), freq="D")

# Build a full grid for each sku_name
sku_names = pilot["sku_name"].unique()
grid = pd.MultiIndex.from_product([full_dates, sku_names], names=["date", "sku_name"]).to_frame(index=False)

# Aggregate in case there are duplicates (safety)
agg = (pilot
       .groupby(["date", "sku_name"], as_index=False)["sales"]
       .sum())

# Left join onto the full grid and fill missing sales with 0
clean = (grid
         .merge(agg, on=["date", "sku_name"], how="left")
         .fillna({"sales": 0}))

clean["store"] = STORE_ID

# Sort
clean = clean.sort_values(["sku_name", "date"]).reset_index(drop=True)

print("Clean shape:", clean.shape)
clean.head()

Clean shape: (9130, 4)


,date,sku_name,sales,store
0,2013-01-01,Effortless Pant,49,1
1,2013-01-02,Effortless Pant,39,1
2,2013-01-03,Effortless Pant,57,1
3,2013-01-04,Effortless Pant,46,1
4,2013-01-05,Effortless Pant,45,1


In [7]:
#Quick quality checks
# 5 series?
print("Number of SKUs:", clean["sku_name"].nunique())

# All dates same length per SKU?
counts = clean.groupby("sku_name")["date"].nunique()
print(counts)

# Any negative sales?
if (clean["sales"] < 0).any():
    raise ValueError("Found negative sales values — investigate.")

Number of SKUs: 5
sku_name
Effortless Pant           1826
Sculpt Knit Tank          1826
Super Puff Shorty         1826
TNA Sweatfleece Hoodie    1826
Wilfred Sweater           1826
Name: date, dtype: int64
